In [2]:
import tensorflow as tf
from tensorflow.keras import models
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False
import os
import numpy as np
from PIL import Image
import time
import functools
import tensorflow as tf
# import tensorflow.contrib.eager as tfe

from tensorflow.keras.preprocessing import image as kp_image


from tensorflow.keras.preprocessing.image import load_img

from tensorflow.python.keras import losses
from tensorflow.python.keras import layers
from tensorflow.python.keras import backend as K

from matplotlib.dviread import Tfm

In [4]:
def load_img(path_to_img):
  max_dim = 512
  img = Image.open(path_to_img)
  long = max(img.size)
  scale = max_dim/long
  img = img.resize((round(img.size[0]*scale), round(img.size[1]*scale)), Image.ANTIALIAS)
  
  img = kp_image.img_to_array(img)
  
  # We need to broadcast the image array such that it has a batch dimension 
  img = np.expand_dims(img, axis=0)
  return img

In [5]:
def imshow(img, title=None):
  # Remove the batch dimension
  out = np.squeeze(img, axis=0)
  # Normalize for display 
  out = out.astype('uint8')
  plt.imshow(out)
  if title is not None:
    plt.title(title)
  plt.imshow(out)

In [6]:
# Content layer where will pull our feature maps
content_layers = ['conv5_block1_1_bn'] 

# Style layer we are interested in
style_layers = ['conv5_block1_1_bn','conv2_block1_2_bn'#,'conv1'
               ]

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)
def get_model(style_layers, content_layers):
    """Creates a model that outputs intermediate layers of ResNet50.
    
    Args:
        style_layers: List of layer names for style representation.
        content_layers: List of layer names for content representation.
        
    Returns:
        A Keras model that takes image inputs and outputs the specified intermediate layers.
    """
    # Load the pre-trained ResNet50 model
    resnet = tf.keras.applications.ResNet50(include_top=False, weights='imagenet')
    resnet.trainable = False
    
    # Get the output layers corresponding to style and content layers
    style_outputs = [resnet.get_layer(name).output for name in style_layers]
    content_outputs = [resnet.get_layer(name).output for name in content_layers]
    
    # Concatenate the outputs for both style and content layers
    model_outputs = style_outputs + content_outputs
    
    # Define the model with ResNet50 input and the selected intermediate layers as outputs
    model = models.Model(resnet.input, model_outputs)
    
    return model


In [7]:
def load_and_process_img(path):
    img = Image.open(path)
    img = np.array(img)
    img = tf.keras.applications.resnet50.preprocess_input(img)
    img = tf.convert_to_tensor(img)
    img = tf.expand_dims(img, axis=0)
    return img

In [8]:
def get_feature_representations(model, content_path, style_path):
    """Helper function to compute our content and style feature representations.

    This function will simply load and preprocess both the content and style 
    images from their path. Then it will feed them through the network to obtain
    the outputs of the intermediate layers. 
    
    Arguments:
        model: The model that we are using.
        content_path: The path to the content image.
        style_path: The path to the style image
        
    Returns:
        Returns the style features and the content features. 
    """
    # Load our images
    content_image = load_and_process_img(content_path)
    style_image = load_and_process_img(style_path)
    
    # Batch compute content and style features
    style_outputs = model(style_image)
    content_outputs = model(content_image)
    
    # Get the style and content feature representations from our model
    style_features = [style_layer[0] for style_layer in style_outputs[:num_style_layers]]
    content_features = [content_layer[0] for content_layer in content_outputs[:num_style_layers]]  # Corrected indexing
    
    return style_features, content_features


In [9]:
def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
  """This function will compute the loss total loss.
  
  Arguments:
    model: The model that will give us access to the intermediate layers
    loss_weights: The weights of each contribution of each loss function. 
      (style weight, content weight, and total variation weight)
    init_image: Our initial base image. This image is what we are updating with 
      our optimization process. We apply the gradients wrt the loss we are 
      calculating to this image.
    gram_style_features: Precomputed gram matrices corresponding to the 
      defined style layers of interest.
    content_features: Precomputed outputs from defined content layers of 
      interest.
      
  Returns:
    returns the total loss, style loss, content loss, and total variational loss
  """
  style_weight, content_weight = loss_weights
  
  # Feed our init image through our model. This will give us the content and 
  # style representations at our desired layers. Since we're using eager
  # our model is callable just like any other function!
  model_outputs = model(init_image)
  
  style_output_features = model_outputs[:num_style_layers]
  content_output_features = model_outputs[num_style_layers:]
  
  style_score = 0
  content_score = 0

  # Accumulate style losses from all layers
  # Here, we equally weight each contribution of each loss layer
  weight_per_style_layer = 1.0 / float(num_style_layers)
  for target_style, comb_style in zip(gram_style_features, style_output_features):
    style_score += weight_per_style_layer * get_style_loss(comb_style[0], target_style)
    
  # Accumulate content losses from all layers 
  weight_per_content_layer = 1.0 / float(num_content_layers)
  for target_content, comb_content in zip(content_features, content_output_features):
    content_score += weight_per_content_layer* get_content_loss(comb_content[0], target_content)
  
  style_score *= style_weight
  content_score *= content_weight

  # Get total loss
  loss = style_score + content_score 
  return loss, style_score, content_score

In [10]:
def get_content_loss(base_content, target):
  return tf.reduce_mean(tf.square(base_content - target))

In [11]:
def gram_matrix(input_tensor):
  # We make the image channels first 
  channels = int(input_tensor.shape[-1])
  a = tf.reshape(input_tensor, [-1, channels])
  n = tf.shape(a)[0]
  gram = tf.matmul(a, a, transpose_a=True)
  return gram / tf.cast(n, tf.float32)

def get_style_loss(base_style, gram_target):
  """Expects two images of dimension h, w, c"""
  # height, width, num filters of each layer
  # We scale the loss at a given layer by the size of the feature map and the number of filters
  height, width, channels = base_style.get_shape().as_list()
  gram_style = gram_matrix(base_style)
  
  return tf.reduce_mean(tf.square(gram_style - gram_target))# / (4. * (channels ** 2) * (width * height) ** 2)

In [12]:
def deprocess_img(processed_img):
    x = processed_img.numpy()
    x = x.squeeze(0)
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [14]:
def compute_grads(cfg):
  with tf.GradientTape() as tape: 
    all_loss = compute_loss(**cfg)
  # Compute gradients wrt input image
  total_loss = all_loss[0]
  return tape.gradient(total_loss, cfg['init_image']), all_loss

In [15]:
def run_style_transfer(content_path, style_path, model_weights_path):
    # Load the pre-trained model
    model = get_model(style_layers, content_layers)
    model.load_weights(model_weights_path)

    # Load and preprocess content and style images
    content_image = load_and_process_img(content_path)
    style_image = load_and_process_img(style_path)

    # Forward pass through the model to get the feature maps for both content and style images
    content_features = model(content_image)
    style_features = model(style_image)

    # Concatenate the feature maps
    concatenated_features = tf.concat([content_features, style_features], axis=-1)

    # Run the style transfer process on the concatenated feature maps
    stylized_features = style_transfer_process(concatenated_features)

    # Deprocess the stylized feature maps
    stylized_image = deprocess_img(stylized_features)

    return stylized_image



# Paths to content and style images
content_path = "input_image.jpg"
style_path = "style.jpg"

# Path to pre-trained model weights
model_weights_path = "model_weights.weights.h5"

# Run style transfer
stylized_image = run_style_transfer(content_path, style_path, model_weights_path)

# Display the stylized image
plt.imshow(stylized_image)
plt.axis('off')
plt.show()

KeyboardInterrupt: 